<a href="https://colab.research.google.com/github/milanschroeder/TRIAS-paper1/blob/main/11_digitality_zs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zeroshot Classification: Digital Sub-Topics

In [4]:
# load modules:
import pandas as pd
import numpy as np
import time
import os

# load data:
output_path = '/content/drive/MyDrive/TRIAS/bart_zs.csv'
input_path = '/content/drive/MyDrive/TRIAS/all_unique_texts.csv' # with id, text, target
df = pd.read_csv(input_path)

### if need to restart (due to Colab Timeout):
#classified_communication = pd.read_csv(output_path)
#df = df[~df.index.isin(classified_communication.index)]

In [5]:
# have a look at the data
df

,Unnamed: 0,id,text,digitality_sum,obs,overlap,digital,definition_zeroshot,positive,negative,...,contradiction,digital_communications,internet_technologies,digital_services_algorithms,digitized_data,algorithm,online,computer,internet,target
0,0,28.0,The regional taskforce will concentrate on the...,0.0,1,False,0.187055,0.899664,0.712950,0.287050,...,0.029479,0.282811,0.466247,0.376654,0.276695,0.192682,0.819111,0.404290,0.127403,False
1,1,33.0,It is my pleasure to inform you about our firs...,0.0,1,False,0.194947,0.849398,0.897798,0.102202,...,0.005455,0.075267,0.089725,0.052692,0.047672,0.091960,0.545678,0.197250,0.041259,False
2,2,45.0,The pandemic has only strengthened the case fo...,0.0,1,False,0.136782,0.785451,0.723874,0.276126,...,0.046951,0.089978,0.236516,0.099237,0.108618,0.022648,0.793313,0.041527,0.197752,False
3,3,53.0,This discussion also included a first factual ...,0.0,1,False,0.103987,0.883559,0.616180,0.383820,...,0.036785,0.005410,0.009913,0.004431,0.021108,0.018592,0.932250,0.070977,0.084058,False
4,4,55.0,"Then, High Representative/ Vice President of t...",0.0,1,False,0.115152,0.843238,0.897171,0.102829,...,0.003368,0.069245,0.191867,0.100428,0.073180,0.059256,0.488432,0.120646,0.035329,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2732,2732,2106305.0,IMPROVEMENT OF AGRICULTURAL INCOMES IN THE COM...,0.0,1,False,0.009425,0.891605,0.913306,0.086694,...,0.012628,0.001192,0.001489,0.002471,0.018417,0.004602,0.295374,0.000625,0.002018,False
2733,2733,2107628.0,INTEGRATED MEDITERRANEAN PROGRAMME FOR WESTERN...,0.0,1,False,0.338812,0.889597,0.924641,0.075359,...,0.005481,0.020785,0.172198,0.761176,0.871382,0.637480,0.920099,0.471818,0.004481,False
2734,2734,2107681.0,SUMMARY OF SPEECH FOR MR SUTHERLAND TO A MEETI...,0.0,1,False,0.062550,0.802197,0.902336,0.097664,...,0.007649,0.036708,0.025715,0.019737,0.020600,0.002132,0.773107,0.032268,0.011590,False
2735,2735,2107847.0,EUROPEAN YEAR OF THE ENVIRONMENT : BETTER ENVI...,0.0,1,False,0.160889,0.853055,0.911146,0.088854,...,0.009711,0.011525,0.022864,0.069323,0.082779,0.030434,0.790829,0.107925,0.029455,False


# Zeroshot Classification:

In [8]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
new_data = df[:0]

# Record the start time
start_time = time.time()
progress = 0

hypotheses = [
      "This text is about digital communications.",
      "This text relates to internet technologies.",
      "This text involves digital services and algorithms.",
      "This text discusses digitized data.",
      "This text is about digital policy."
  ]

label = [
    'digital_communications',
    'internet_technologies',
    'digital_services_algorithms',
    'digitized_data',
    'digital_policy'
]

for i, row in df.iterrows():

  premise = row['text']

  result = classifier(premise, hypotheses, multi_label = True)

  # Extracting the scores
  scores = result['scores']
  labels = result['labels']


  for j in hypotheses:
    df.at[i, label[j]] = scores[labels.index(hypotheses[j])]

  new_data = pd.concat([new_data, pd.DataFrame(df.loc[i]).T])

  progress = progress + 1

  if progress % 100 == 0:
    end_time = time.time()
    mean_time = end_time - start_time
    print(progress, "after", mean_time)

    # save new_data:
    new_data.to_csv(output_path, mode='a', header=not os.path.exists(output_path), index=False) #append df every 1000 classifications
    new_data = new_data[:0] # truncate again

# save last new_data:
if progress % 100 != 0:
  new_data.to_csv(output_path, mode='a', header=not os.path.exists(output_path), index=False)

end_time = time.time()
total_time = end_time - start_time

print("\nTotal time:", total_time, "\nTotal classifications:", progress, "\nmean time per classification:", total_time/progress)

100 after 578.2488884925842
200 after 1051.6790914535522
300 after 1508.0873346328735
400 after 1985.663603067398
500 after 2566.277529001236
600 after 3056.0160524845123
700 after 3585.226594209671
800 after 4058.6199526786804
900 after 4533.587757825851
1000 after 5310.818298816681
1100 after 6105.753247976303
1200 after 6986.654340267181
1300 after 7300.719390630722
1400 after 7630.794435977936
1500 after 7958.362741231918

Total time: 8079.220733880997 
Total classifications: 1537 
mean time per classification: 5.256487139805463
